<img src=./imgs/model_io.jpg width=35% />

[langchain documents](https://python.langchain.com/docs/modules/model_io/models/chat/llm_chain)

[LangChain-Tutorials](https://github.com/sugarforever/LangChain-Tutorials)

In [1]:
import os
os.environ['OPENAI_API_KEY'] = '&&&&&&&&&&'

# Select by length

>  LLM的输入长度是有限的<br>

In [2]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate

from langchain.prompts.example_selector import LengthBasedExampleSelector

In [3]:
# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"}]

In [4]:
example_prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template="Input: {input}\nOutput: {output}"
)

In [5]:
example_selector = LengthBasedExampleSelector(
    # these are the examples it has available to choose from。
    examples=examples,
    # this is the prompt template being used to format examples
    example_prompt=example_prompt,
    # this is the maximum length that the formatted examples should be.
    # length is measured by the get_text_length function below.
    max_length=25,
    
    # this is the function used to get the length of a string.
    # which is used to determine which example to include。
    # it is commented out because it is provided by a default value if None is specified
    # 他被注释掉了，因为如果不指名，它将作为默认值使用
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)

In [6]:
dynamic_prompt = FewShotPromptTemplate(
    # we provided an exampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix='Give the antonym of every input',
    suffix='Input: {adjective}\nOutput:',
    input_variables=['adjective']
)

In [7]:
# An example with small input, so it select all examples.
print(dynamic_prompt.format(adjective='big'))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [8]:
# An example with long input, so it selects only on example.
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"

In [9]:
len(long_string)

113

In [11]:
prompt = dynamic_prompt.format(adjective=long_string)

In [12]:
len(prompt)

187

In [14]:
import re

In [18]:
prompt_remove_blank = re.split("\n| ", prompt)
prompt_remove_blank

['Give',
 'the',
 'antonym',
 'of',
 'every',
 'input',
 '',
 'Input:',
 'happy',
 'Output:',
 'sad',
 '',
 'Input:',
 'big',
 'and',
 'huge',
 'and',
 'massive',
 'and',
 'large',
 'and',
 'gigantic',
 'and',
 'tall',
 'and',
 'much',
 'much',
 'much',
 'much',
 'much',
 'bigger',
 'than',
 'everything',
 'else',
 'Output:']

In [19]:
len(prompt_remove_blank)

35

In [27]:
re.split("\n| ", "Input: {input}\nOutput: {output}".format(input=long_string, output=''))

['Input:',
 'big',
 'and',
 'huge',
 'and',
 'massive',
 'and',
 'large',
 'and',
 'gigantic',
 'and',
 'tall',
 'and',
 'much',
 'much',
 'much',
 'much',
 'much',
 'bigger',
 'than',
 'everything',
 'else',
 'Output:',
 '']

In [28]:
re.split("\n| ", "Input: {input}\nOutput: {output}".format(input=long_string, output='')).__len__()

24

> <font color=green size=5>是一个example的长度不能超过25 <br>  我们试着让他超过25试试</font>

In [29]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else big big big big big"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else big big big big big
Output:


> <font color=green size=5>他连第一个example 都顶掉了</font>

In [13]:
print(prompt)

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [20]:
# you can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)


print(dynamic_prompt.format(adjective='enthusiastic'))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output: small

Input: enthusiastic
Output:
